Imports

In [12]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from os import path, getcwd


Open file, create training sets

In [8]:
# path is ../sql/db/__NAMEOFCSV__.csv
pathToParent = path.dirname(os.getcwd())
csvFile = pathToParent + "/sqlite/db/DummyFintech.csv"
completeDF = pd.read_csv(csvFile, header = 0)

trainDF, testDF = train_test_split(completeDF, test_size=.2, random_state=435)


trainX = trainDF.drop(columns=['dayafter', 'monthafter', 'threemonthsafter'])

long_y_train = trainDF['threemonthsafter']
medium_y_train = trainDF['monthafter']
short_y_train = trainDF['dayafter']

testX = testDF.drop(columns=['dayafter', 'monthafter', 'threemonthsafter'])

long_y_test = testDF['threemonthsafter']
medium_y_test = testDF['monthafter']
short_y_test = testDF['dayafter']



In [9]:
print(long_y_test)
print(medium_y_test)
print(short_y_test)
print("NEXT \n")
print(short_y_train)
print(trainX)

4    4.0
Name: threemonthsafter, dtype: float64
4    3.0
Name: monthafter, dtype: float64
4    2.0
Name: dayafter, dtype: float64
NEXT 

0    2.0
3    2.0
1    2.0
2    2.0
Name: dayafter, dtype: float64
         date  daybefore  transcript
0  1681147907        1.0  transcript
3  1681147907        1.0  transcript
1  1681147907        1.0  transcript
2  1681147907        1.0  transcript


Create Dummy models

In [10]:
shortDummy = DummyClassifier()
mediumDummy = DummyClassifier()
longDummy = DummyClassifier()

shortDummy.fit(trainX, short_y_train)
mediumDummy.fit(trainX, medium_y_train)
longDummy.fit(trainX, long_y_train)

print(shortDummy.predict(trainX))
print(mediumDummy.predict(testX))
print(longDummy.predict(testX))

[2. 2. 2. 2.]
[3.]
[4.]


Create Test and trianing sets

Create preprocessing

Test different models

Creating a grid search to find best hyperperameters. 

Create predictions

Add predictions to database